## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-12-09-30-00 +0000,wsj,Pittsburgh is striving to turn its old steel m...,https://www.wsj.com/tech/ai/can-pittsburghs-ol...
1,2025-07-12-09-25-13 +0000,bbc,Two Palestinians killed in West Bank settler a...,https://www.bbc.com/news/articles/cm2mknzn99eo
2,2025-07-12-09-15-28 +0000,nypost,US sanctions Cuban President Díaz-Canel and ot...,https://nypost.com/2025/07/12/us-news/us-sanct...
3,2025-07-12-09-00-22 +0000,nyt,A Landscape of Death: What’s Left Where Ukrain...,https://www.nytimes.com/2025/07/12/world/europ...
4,2025-07-12-08-35-29 +0000,bbc,What we know so far about Air India crash inve...,https://www.bbc.com/news/articles/c5y5nq170z4o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2366/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
178,trump,19
297,canada,9
350,tariff,6
34,ukraine,5
420,latest,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
55,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...,47
81,2025-07-11-13-41-59 +0000,cbc,CUSMA-compliant goods exempt from Trump's late...,https://www.cbc.ca/news/business/trump-tariff-...,46
43,2025-07-11-22-51-48 +0000,nypost,Dow drops nearly 300 points after Trump teases...,https://nypost.com/2025/07/11/us-news/dow-drop...,44
36,2025-07-11-23-48-26 +0000,nypost,Judge releases bizarre letter from would-be Tr...,https://nypost.com/2025/07/11/us-news/judge-re...,42
21,2025-07-12-02-08-29 +0000,nypost,Trump grants ICE ability to arrest ‘SLIMEBALL’...,https://nypost.com/2025/07/11/us-news/trump-gr...,41


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
55,47,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...
28,26,2025-07-12-00-29-57 +0000,nypost,Migrant farm worker dies after falling 30 feet...,https://nypost.com/2025/07/11/us-news/migrant-...
19,23,2025-07-12-02-41-31 +0000,nypost,Texas family mourns ‘shining light’ 8-year-old...,https://nypost.com/2025/07/11/us-news/texas-fa...
36,23,2025-07-11-23-48-26 +0000,nypost,Judge releases bizarre letter from would-be Tr...,https://nypost.com/2025/07/11/us-news/judge-re...
76,22,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
6,20,2025-07-12-08-00-00 +0000,cbc,Poilievre says he lost Ottawa riding for being...,https://www.cbc.ca/news/politics/poilievre-pub...
22,19,2025-07-12-01-53-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...
11,16,2025-07-12-04-50-49 +0000,nypost,"Alleged Florida child rapist lured girl, 11, i...",https://nypost.com/2025/07/12/us-news/alleged-...
65,16,2025-07-11-17-37-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...
31,15,2025-07-12-00-13-34 +0000,nypost,Zohran Mamdani said NYPD shouldn’t respond to ...,https://nypost.com/2025/07/11/us-news/zohran-m...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
